In [84]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.holiday import Holiday, AbstractHolidayCalendar
from dateutil.easter import easter
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [85]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [88]:
test_data = pd.read_parquet(Path("data") / "final_test.parquet")

In [89]:
class FrenchHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday("New Year's Day", month=1, day=1),
        Holiday("Labour Day", month=5, day=1),
        Holiday("Victory in Europe Day", month=5, day=8),
        Holiday("Bastille Day", month=7, day=14),
        Holiday("Assumption of Mary", month=8, day=15),
        Holiday("All Saints' Day", month=11, day=1),
        Holiday("Armistice Day", month=11, day=11),
        Holiday("Christmas Day", month=12, day=25),
    ]

    @staticmethod
    def easter_related_holidays(year):
        easter_sunday = easter(year)
        return [
            (easter_sunday + timedelta(days=1), "Easter Monday"),
            (easter_sunday + timedelta(days=39), "Ascension Day"),
        ]

In [90]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X["date"] = X.index

    # Encode the date information
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Boolean feature for weekends
    X.loc[:, "is_weekend"] = X["weekday"].isin(
        [5, 6]
    )  # 5 and 6 correspond to Saturday and Sunday

    # Boolean feature for holidays
    cal = FrenchHolidayCalendar()
    holidays = cal.holidays(start=X["date"].min(), end=X["date"].max())

    # Add Easter related holidays
    easter_holidays = []
    for year in range(X["date"].dt.year.min(), X["date"].dt.year.max() + 1):
        for date, _ in FrenchHolidayCalendar.easter_related_holidays(year):
            easter_holidays.append(date)

    holidays = holidays.union(pd.to_datetime(easter_holidays))
    X.loc[:, "is_holiday"] = X["date"].isin(holidays)

    return X.drop(columns=["date"])

In [91]:
def cyclical_encode(df, column, max_value):
    df[column + "_sin"] = np.sin(2 * np.pi * df[column] / max_value)
    df[column + "_cos"] = np.cos(2 * np.pi * df[column] / max_value)
    return df


def encode_cyclical_features(X):
    X = X.copy()

    X = cyclical_encode(X, "hour", 24)

    X = cyclical_encode(X, "weekday", 7)

    return X

In [92]:
def encode_lockdown_periods(X):
    X = X.copy()
    X["date"] = X.index

    # Define lockdown periods
    lockdowns = {
        "lockdown_1": ("2020-03-17", "2020-05-10"),
        "lockdown_2": ("2020-10-28", "2020-12-01"),
        # with curfew from 7 PM to 6 AM
        "lockdown_3_1": ("2021-04-03", "2021-05-18"),
        # with curfew from 9 PM to 6 AM
        "lockdown_3_2": ("2021-05-19", "2021-06-08"),
        # with curfew from 11 PM to 6 AM
        "lockdown_3_3": ("2021-06-09", "2021-06-29"),
    }

    # Initialize lockdown columns to False
    for lockdown in lockdowns:
        X[lockdown] = False

    # Mark the lockdown periods
    for lockdown, (start_date, end_date) in lockdowns.items():
        mask = (X["date"] >= start_date) & (X["date"] <= end_date)
        X.loc[mask, lockdown] = True

    return X.drop(columns=["date"])

In [93]:
weather_data = pd.read_csv("data/external_data_cleaned.csv")

In [94]:
# Drop the 'week' and 'day' columns
weather_data = weather_data.drop(columns=["week", "day"])

In [95]:
data["date"] = pd.to_datetime(data["date"])
weather_data["date"] = pd.to_datetime(weather_data["date"])
test_data["date"] = pd.to_datetime(test_data["date"])

In [96]:
data.set_index("date", inplace=True)
weather_data.set_index("date", inplace=True)
test_data.set_index("date", inplace=True)

In [97]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3321 entries, 2021-01-01 00:00:00 to 2020-09-30 21:00:00
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   t              3321 non-null   float64
 1   rr1            3321 non-null   float64
 2   rr3            3321 non-null   float64
 3   rr6            3321 non-null   float64
 4   ff             3321 non-null   float64
 5   raf10          3321 non-null   float64
 6   rafper         3321 non-null   float64
 7   u              3321 non-null   int64  
 8   vv             3321 non-null   int64  
 9   n              3321 non-null   float64
 10  cl             3321 non-null   float64
 11  cm             3321 non-null   float64
 12  ch             3321 non-null   float64
 13  precipitation  3321 non-null   int64  
 14  cloudy_day     3321 non-null   int64  
dtypes: float64(11), int64(4)
memory usage: 415.1 KB


In [98]:
weather_data_hourly = weather_data.resample("H").ffill()

In [99]:
combined_data = data.merge(
    weather_data_hourly, left_on="date", right_index=True, how="inner"
)

In [100]:
combined_test_data = test_data.merge(
    weather_data_hourly, left_on="date", right_index=True, how="inner"
)

In [101]:
combined_test_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51440 entries, 2021-09-10 01:00:00 to 2021-09-27 11:00:00
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   counter_id                 51440 non-null  category      
 1   counter_name               51440 non-null  category      
 2   site_id                    51440 non-null  int64         
 3   site_name                  51440 non-null  category      
 4   counter_installation_date  51440 non-null  datetime64[ns]
 5   coordinates                51440 non-null  category      
 6   counter_technical_id       51440 non-null  category      
 7   latitude                   51440 non-null  float64       
 8   longitude                  51440 non-null  float64       
 9   t                          51440 non-null  float64       
 10  rr1                        51440 non-null  float64       
 11  rr3                        51440

In [102]:
combined_data = _encode_dates(combined_data)

In [103]:
combined_test_data = _encode_dates(combined_test_data)

In [104]:
combined_data = encode_cyclical_features(combined_data)
combined_test_data = encode_cyclical_features(combined_test_data)

In [105]:
combined_data = encode_lockdown_periods(combined_data)
combined_test_data = encode_lockdown_periods(combined_test_data)

In [106]:
combined_data.drop(columns=["bike_count", "counter_id", "site_id"], inplace=True)

In [107]:
# Creating 'counter_age' column
combined_data["counter_installation_date"] = pd.to_datetime(
    combined_data["counter_installation_date"]
)

In [108]:
combined_data["counter_age"] = (
    combined_data.index - combined_data["counter_installation_date"]
).dt.days
combined_data.drop(columns=["counter_installation_date"], inplace=True)

In [110]:
# Creating lag and rolling features
for lag in [1, 2, 7]:  # lags of 1 day, 2 days, and 1 week
    combined_data[f"log_bike_count_lag_{lag}"] = combined_data["log_bike_count"].shift(
        lag
    )

In [111]:
for lag in [1, 2, 7]:
    combined_data[f"log_bike_count_lag_{lag}_squared"] = (
        combined_data[f"log_bike_count_lag_{lag}"] ** 2
    )
    combined_data[f"log_bike_count_lag_{lag}_cubed"] = (
        combined_data[f"log_bike_count_lag_{lag}"] ** 3
    )

In [112]:
# 7-day rolling average
combined_data["log_bike_count_rolling_7"] = (
    combined_data["log_bike_count"].rolling(window=7).mean()
)

In [113]:
# Dropping NaN values created by lag and rolling features
combined_data.dropna(inplace=True)

In [114]:
# Dropping or keeping latitude and longitude
# combined_data.drop(columns=["latitude", "longitude"], inplace=True)

In [115]:
bool_columns = [
    "is_weekend",
    "is_holiday",
    "lockdown_1",
    "lockdown_2",
    "lockdown_3_1",
    "lockdown_3_2",
    "lockdown_3_3",
]
combined_data[bool_columns] = combined_data[bool_columns].astype(int)

In [116]:
# one-hot-encoding
categorical_columns = ["counter_name", "site_name", "counter_technical_id"]

combined_data_encoded = pd.get_dummies(
    combined_data, columns=categorical_columns)

In [117]:
# splitting data

split_ratio = 0.8
split_index = int(len(combined_data_encoded) * split_ratio)

X = combined_data_encoded.drop(columns=["log_bike_count"])
y = combined_data_encoded["log_bike_count"]

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Linear Regression for capturing trend

In [118]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_trend_train = lr_model.predict(X_train)
lr_trend_test = lr_model.predict(X_test)

# XGBoost on residuals

In [119]:
residuals_train = y_train - lr_trend_train
residuals_test = y_test - lr_trend_test

xgb_model = XGBRegressor()

In [120]:
xgb_model.fit(X_train, residuals_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [121]:
xgb_residuals_train = xgb_model.predict(X_train)
xgb_residuals_test = xgb_model.predict(X_test)

# Combine predictions:
final_predictions_train = lr_trend_train + xgb_residuals_train
final_predictions_test = lr_trend_test + xgb_residuals_test

In [122]:
rmse = mean_squared_error(y_test, final_predictions_test, squared=False)
mae = mean_absolute_error(y_test, final_predictions_test)
print(f"RMSE: {rmse}, MAE: {mae}")

RMSE: 0.47213568069551176, MAE: 0.30230427810097044


# XGBoost in general

In [69]:
xgb_model = XGBRegressor()

In [70]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [71]:
y_pred = xgb_model.predict(X_test)

In [72]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.6741174340531096


# processing test data

In [123]:
combined_test_data.drop(columns=["counter_id", "site_id"], inplace=True)

In [124]:
# Creating 'counter_age' column
combined_test_data["counter_installation_date"] = pd.to_datetime(
    combined_test_data["counter_installation_date"]
)

In [125]:
combined_test_data["counter_age"] = (
    combined_test_data.index - combined_test_data["counter_installation_date"]
).dt.days
combined_test_data.drop(columns=["counter_installation_date"], inplace=True)

# Creating lag and rolling features
for lag in [1, 2, 7]:  # lags of 1 day, 2 days, and 1 week
    combined_test_data[f"log_bike_count_lag_{lag}"] = combined_test_data[
        "log_bike_count"
    ].shift(lag)

for lag in [1, 2, 7]:  # Assuming you have these lags
    combined_test_data[f"log_bike_count_lag_{lag}_squared"] = (
        combined_test_data[f"log_bike_count_lag_{lag}"] ** 2
    )
    combined_test_data[f"log_bike_count_lag_{lag}_cubed"] = (
        combined_test_data[f"log_bike_count_lag_{lag}"] ** 3
    )
    # combined_test_data[f"log_bike_count_lag_{lag}_order_4"] = (
    #    combined_test_data[f"log_bike_count_lag_{lag}"] ** 4
    # )
    # combined_test_data[f"log_bike_count_lag_{lag}_order_5"] = (
    #    combined_test_data[f"log_bike_count_lag_{lag}"] ** 5
    # )

# 7-day rolling average
combined_test_data["log_bike_count_rolling_7"] = (
    combined_test_data["log_bike_count"].rolling(window=7).mean()
)

In [76]:
# Dropping NaN values created by lag and rolling features
combined_test_data.dropna(inplace=True)

In [127]:
# Dropping or keeping latitude and longitude
# combined_test_data.drop(columns=["latitude", "longitude"], inplace=True)

In [128]:
bool_columns = [
    "is_weekend",
    "is_holiday",
    "lockdown_1",
    "lockdown_2",
    "lockdown_3_1",
    "lockdown_3_2",
    "lockdown_3_3",
]
combined_test_data[bool_columns] = combined_test_data[bool_columns].astype(int)

In [129]:
# one-hot-encoding
categorical_columns = ["counter_name", "site_name", "counter_technical_id"]

combined_test_data_encoded = pd.get_dummies(
    combined_test_data, columns=categorical_columns
)

In [130]:
combined_test_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51440 entries, 2021-09-10 01:00:00 to 2021-09-27 11:00:00
Columns: 151 entries, coordinates to counter_technical_id_YTH19111510
dtypes: category(1), float64(17), int32(7), int64(10), uint8(116)
memory usage: 18.1 MB


In [81]:
X_test_data = combined_test_data_encoded.drop(columns=["log_bike_count"])
y_test_data = combined_test_data_encoded["log_bike_count"]

In [ ]:
lr_trend_test_data = lr_model.predict(X_test_data)

In [ ]:
residuals_test_data = y_test_data - lr_trend_test_data

In [ ]:
xgb_residuals_test_data = xgb_model.predict(X_test_data)

In [ ]:
final_predictions_test_data = lr_trend_test_data + xgb_residuals_test_data

In [ ]:
rmse_test = mean_squared_error(
    y_test_data, final_predictions_test_data, squared=False)
mae_test = mean_absolute_error(y_test_data, final_predictions_test_data)
print(f"RMSE: {rmse_test}, MAE: {mae_test}")

In [82]:
y_test_pred = xgb_model.predict(X_test_data)

In [83]:
rmse_test = mean_squared_error(y_test_data, y_test_pred, squared=False)
print(f"RMSE: {rmse_test}")

RMSE: 0.9156888716305859


If test data is unseen

In [131]:
unseen_data = combined_test_data_encoded

In [139]:
unseen_data = unseen_data.drop("coordinates", axis=1)

In [140]:
col_names = unseen_data.columns.tolist()

In [142]:
rolling_predictions = []  # to keep track of the last 7 predictions for rolling average

In [143]:
for i in range(len(unseen_data)):
    # Prepare the features for prediction
    current_features = unseen_data.iloc[i].values.reshape(1, -1)

    # Predict the trend using the linear regression model
    trend_prediction = lr_model.predict(current_features)[0]

    # Predict the residual using the XGBoost model
    residual_prediction = xgb_model.predict(current_features)[0]

    # Final prediction is the sum of trend and residual
    final_prediction = trend_prediction + residual_prediction
    rolling_predictions.append(final_prediction)

    # Update lagged features for the next prediction
    if i + 1 < len(unseen_data):
        unseen_data.at[i + 1, "log_bike_count_lag_1"] = final_prediction
        if i + 2 < len(unseen_data):
            unseen_data.at[i + 2, "log_bike_count_lag_2"] = final_prediction
        if i + 7 < len(unseen_data):
            unseen_data.at[i + 7, "log_bike_count_lag_7"] = final_prediction

    # Update squared and cubed lag features
    for lag in [1, 2, 7]:
        if i + lag < len(unseen_data):
            unseen_data.at[i + lag, f"log_bike_count_lag_{lag}_squared"] = (
                final_prediction**2
            )
            unseen_data.at[i + lag, f"log_bike_count_lag_{lag}_cubed"] = (
                final_prediction**3
            )

    # Update rolling average
    if len(rolling_predictions) > 7:
        rolling_predictions.pop(0)  # keep only the last 7 predictions
    if i + 1 < len(unseen_data):
        unseen_data.at[i + 1, "log_bike_count_rolling_7"] = np.mean(rolling_predictions)

c:\Users\ameya\miniforge-pypy3\envs\bikes-count\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 150 features, but LinearRegression is expecting 160 features as input.